In [1]:
#environment setup  C:\Users\Administrator\Desktop\simple eda\simple eda\games_table.ipynb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

original_file="C:\\Users\\Administrator\\Desktop\\2014.10-15.2\\results\\aftermerge\\1410-1503.csv"
df=pd.read_csv(original_file)

In [2]:
import math
import chess
import os
import sys
import chess.pgn
import chess.engine
import csv
import io
import traceback
import subprocess
import time
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
import asyncio
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
from stockfish import Stockfish  
#强制更换 event loop（解决部分 Windows 异步问题）
if sys.platform.startswith('win'):
    asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
#environment setup
import tqdm
import numpy as np

In [3]:
def get_eval_str(score, board):
    #score is mate
    if isinstance(score, chess.engine.Mate):
        mate_value = score.mate()
        if (mate_value > 0 and board.turn == chess.WHITE) or (mate_value < 0 and board.turn == chess.BLACK):
            mating_side = "White"
        else:
            mating_side = "Black"
        return f"Mate in {abs(mate_value)} for {mating_side}"
    else:
        #score is centipawn
        cp_score = score.score()
        return f"{cp_score/100.0:.2f}"


def move_accuracy_percent(before, after):
    if after >= before:
        return 100.0 #didnt get worse,think it's a perfect move
    else:
        win_diff = before - after
        raw = 103.1668100711649 * math.exp(-0.04354415386753951 * win_diff) + -3.166924740191411
        return max(min(raw + 1, 100), 0)


def winning_chances_percent(cp):
    multiplier = -0.00368208
    chances = 2 / (1 + math.exp(multiplier * cp)) - 1
    return 50 + 50 * max(min(chances, 1), -1)


def harmonic_mean(values):
    n = len(values)
    if n == 0:
        return 0
    reciprocal_sum = sum(1 / x for x in values if x)
    return n / reciprocal_sum if reciprocal_sum else 0


def std_dev(seq):
    if len(seq) == 0:
        return 0.5 
    mean = sum(seq) / len(seq)
    variance = sum((x - mean) ** 2 for x in seq) / len(seq)
    return math.sqrt(variance)


def volatility_weighted_mean(accuracies, win_chances, is_white):
    weights = []
    for i in range(len(accuracies)):
        base_index = i * 2 + 1 if is_white else i * 2 + 2
        start_idx = max(base_index - 2, 0)
        end_idx = min(base_index + 2, len(win_chances) - 1)

        sub_seq = win_chances[start_idx:end_idx+1]
        weight = max(min(std_dev(sub_seq), 12), 0.5)
        weights.append(weight)

    weighted_sum = sum(a*w for a,w in zip(accuracies,weights))
    total_weight = sum(weights)
    weighted_mean = weighted_sum / total_weight if total_weight else 0

    return weighted_mean


class SimpleStockfishEngine:
    def __init__(self, engine_path, threads=1):
        self.engine_path = Path(engine_path)
        if not self.engine_path.exists():
            raise FileNotFoundError(f"引擎路径不存在: {engine_path}")
        
        self.process = subprocess.Popen(
            str(self.engine_path),
            universal_newlines=True,
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            bufsize=1
        )
        
        self._send_command("uci")
        self._wait_for("uciok")
        self._send_command(f"setoption name Threads value {threads}")
        self._send_command("isready")
        self._wait_for("readyok")
    
    def _send_command(self, command):
        self.process.stdin.write(f"{command}\n")
        self.process.stdin.flush()
    
    def _wait_for(self, text, timeout=5.0):
        start_time = time.time()
        while time.time() - start_time < timeout:
            line = self.process.stdout.readline().strip()
            if text in line:
                return line
        raise TimeoutError(f"等待'{text}'超时")
    
    def _read_until_empty(self):
        lines = []
        while True:
            line = self.process.stdout.readline().strip()
            if not line:
                break
            lines.append(line)
        return lines
    
    def analyse(self, board, depth=16):
        fen = board.fen()
        self._send_command(f"position fen {fen}")
        self._send_command(f"go depth {depth}")
        
        score = None
        best_move = None
        
        while True:
            line = self.process.stdout.readline().strip()
            if "bestmove" in line:
                best_move = line.split()[1]
                break
            
            if "score" in line:
                parts = line.split()
                score_idx = parts.index("score")
                score_type = parts[score_idx + 1]
                score_value = int(parts[score_idx + 2])
                
                if score_type == "cp":
                    score = chess.engine.Cp(score_value)
                elif score_type == "mate":
                    score = chess.engine.Mate(score_value)
        
        if score is None:
            score = chess.engine.Cp(0)  
            
        return {"score": score, "pv": [best_move] if best_move else []}
    
    def quit(self):
        self._send_command("quit")
        self.process.terminate()
        try:
            self.process.wait(timeout=5)
        except subprocess.TimeoutExpired:
            self.process.kill()


def get_score_value(score, board, mate_score=1000):
    #convert the stockfish score to numeric value,
    #and for mate, it's +/-10000
    if isinstance(score, chess.engine.Mate):
        mate = score.mate()
        value = mate_score if mate > 0 else -mate_score
        if mate > 0:
            value = mate_score - mate
        else:
            value = -mate_score - mate
    else:  # Cp
        value = score.score()
        
    if board.turn == chess.BLACK:
        value = -value
        
    return value


In [4]:
def process_csv(csv_file, engine, depth, is_verbose):
    all_games = []
    skipped_indices = []
    game_details = []  # 存储每局游戏的详细分析
    
    with open(csv_file, 'r', newline='', encoding='utf-8') as file:
        # 读第一行获取分隔符
        first_line = file.readline().strip()
        file.seek(0)
        
        delimiter = '\t' if '\t' in first_line else ','
        csv_reader = csv.DictReader(file, delimiter=delimiter)
        rows = list(csv_reader)

        pbar = tqdm.tqdm(total=len(rows), desc="分析棋局")

        
        # 使用tqdm创建进度条
        for row_idx, row in enumerate(rows):
            # 获取基本信息
            white_player = row.get('White', 'Unknown')
            black_player = row.get('Black', 'Unknown')
            white_elo = row.get('WhiteElo', 'N/A')
            black_elo = row.get('BlackElo', 'N/A')
            
            if is_verbose:
                print(f'\n—— Analyzing Game {row_idx+1} ——')
                print(f'White: {white_player} ({white_elo}), Black: {black_player} ({black_elo})')
            
            # 分割走法
            moves_str = row.get('Moves', '')
            if not moves_str:
                all_games.append((row_idx, white_player, black_player, None, None, None, None, None))
                skipped_indices.append(row_idx)
                continue
            
            # 检查走法格式
            moves_list = moves_str.split(',')
            has_invalid_moves = any(move_str.strip() and (len(move_str.strip()) < 4 or len(move_str.strip()) > 5) 
                                     for move_str in moves_list)
            
            if has_invalid_moves:
                all_games.append((row_idx, white_player, black_player, None, None, None, None, None))
                skipped_indices.append(row_idx)
                continue
                
            # 初始化参数
            game_acc_white, game_acc_black = [], []
            game_cp_white, game_cp_black = 0, 0
            game_win_chances = []
            
            # 存储每步详细信息
            move_details = []
            
            board = chess.Board()
            move_number = 1
            analysis_success = True
            
            try:
                for move_idx, move_str in enumerate(moves_list):
                    try:
                        move_str = move_str.strip()
                        if not move_str:
                            continue
                        
                        # 尝试解析走法
                        try:
                            move = chess.Move.from_uci(move_str)
                            if move not in board.legal_moves:
                                analysis_success = False
                                break
                        except chess.InvalidMoveError:
                            analysis_success = False
                            break
                            
                        san_move = board.san(move)
                        
                        # 评估走法前后的局面
                        result_before = engine.analyse(board, depth)
                        score_before = get_score_value(result_before["score"], board)
                        board.push(move)
                        result_after = engine.analyse(board, depth)
                        score_after = get_score_value(result_after["score"], board)
                        
                        # 转换为胜率
                        win_before_white = winning_chances_percent(score_before)
                        win_after_white = winning_chances_percent(score_after)
                        game_win_chances.append(win_after_white)
                        
                        if board.turn == chess.WHITE:  # 黑方刚走完
                            win_before = 100 - win_before_white
                            win_after = 100 - win_after_white
                        else:  # 白方刚走完
                            win_before = win_before_white
                            win_after = win_after_white
                            
                        accuracy = move_accuracy_percent(win_before, win_after)
                        
                        # 计算cp loss
                        if board.turn == chess.BLACK:  # 白方刚走完
                            cp_loss = 0 if score_after > score_before else score_before - score_after
                            game_cp_white += cp_loss
                            game_acc_white.append(accuracy)
                            player = "White"
                        else:  # 黑方刚走完
                            cp_loss = 0 if score_after < score_before else score_after - score_before
                            game_cp_black += cp_loss
                            game_acc_black.append(accuracy)
                            player = "Black"
                            
                        # 存储走棋分析详情，而不是打印
                        step_info = {
                            "move_number": (move_idx // 2) + 1,
                            "san_move": san_move,
                            "player": player,
                            "evaluation": get_eval_str(result_after["score"], board),
                            "cp_loss": cp_loss,
                            "accuracy": accuracy,
                            "win_percentage": win_after_white
                        }
                        move_details.append(step_info)
                        
                        if board.turn == chess.WHITE:
                            move_number += 1
                            
                    except Exception as e:
                        if is_verbose:
                            print(f"Game {row_idx+1}: Error processing move {move_str}: {e}")
                        analysis_success = False
                        break
                
                # 计算每局指标
                if analysis_success and (game_acc_white or game_acc_black):
                    # 基本指标计算
                    avg_cp_white = game_cp_white / len(game_acc_white) if game_acc_white else None
                    avg_cp_black = game_cp_black / len(game_acc_black) if game_acc_black else None
                    
                    harmonic_white = harmonic_mean(game_acc_white) if game_acc_white else None
                    weighted_white = volatility_weighted_mean(game_acc_white, game_win_chances, True) if game_acc_white else None
                    final_acc_white = (harmonic_white + weighted_white) / 2 if harmonic_white is not None and weighted_white is not None else None
                    
                    harmonic_black = harmonic_mean(game_acc_black) if game_acc_black else None
                    weighted_black = volatility_weighted_mean(game_acc_black, game_win_chances, False) if game_acc_black else None
                    final_acc_black = (harmonic_black + weighted_black) / 2 if harmonic_black is not None and weighted_black is not None else None
                    
                    # 按阶段分析 - 将移动划分为3个阶段
                    white_moves = [(i, acc) for i, acc in enumerate(game_acc_white)]
                    black_moves = [(i, acc) for i, acc in enumerate(game_acc_black)]
                    
                    # 阶段分析结果
                    stage_analysis = {
                        "beginning": {"white": {"accuracy": None, "std": None}, "black": {"accuracy": None, "std": None}},
                        "middle": {"white": {"accuracy": None, "std": None}, "black": {"accuracy": None, "std": None}},
                        "endgame": {"white": {"accuracy": None, "std": None}, "black": {"accuracy": None, "std": None}}
                    }
                    
                    # 处理白方走棋按阶段划分
                    if white_moves:
                        n_moves = len(white_moves)
                        begin_idx = n_moves // 3
                        mid_idx = 2 * n_moves // 3
                        
                        begin_moves = [move[1] for move in white_moves[:begin_idx]]
                        mid_moves = [move[1] for move in white_moves[begin_idx:mid_idx]]
                        end_moves = [move[1] for move in white_moves[mid_idx:]]
                        
                        if begin_moves:
                            stage_analysis["beginning"]["white"]["accuracy"] = sum(begin_moves) / len(begin_moves)
                            stage_analysis["beginning"]["white"]["std"] = np.std(begin_moves) if len(begin_moves) > 1 else 0
                            
                        if mid_moves:
                            stage_analysis["middle"]["white"]["accuracy"] = sum(mid_moves) / len(mid_moves)
                            stage_analysis["middle"]["white"]["std"] = np.std(mid_moves) if len(mid_moves) > 1 else 0
                            
                        if end_moves:
                            stage_analysis["endgame"]["white"]["accuracy"] = sum(end_moves) / len(end_moves)
                            stage_analysis["endgame"]["white"]["std"] = np.std(end_moves) if len(end_moves) > 1 else 0
                    
                    # 处理黑方走棋按阶段划分
                    if black_moves:
                        n_moves = len(black_moves)
                        begin_idx = n_moves // 3
                        mid_idx = 2 * n_moves // 3
                        
                        begin_moves = [move[1] for move in black_moves[:begin_idx]]
                        mid_moves = [move[1] for move in black_moves[begin_idx:mid_idx]]
                        end_moves = [move[1] for move in black_moves[mid_idx:]]
                        
                        if begin_moves:
                            stage_analysis["beginning"]["black"]["accuracy"] = sum(begin_moves) / len(begin_moves)
                            stage_analysis["beginning"]["black"]["std"] = np.std(begin_moves) if len(begin_moves) > 1 else 0
                            
                        if mid_moves:
                            stage_analysis["middle"]["black"]["accuracy"] = sum(mid_moves) / len(mid_moves)
                            stage_analysis["middle"]["black"]["std"] = np.std(mid_moves) if len(mid_moves) > 1 else 0
                            
                        if end_moves:
                            stage_analysis["endgame"]["black"]["accuracy"] = sum(end_moves) / len(end_moves)
                            stage_analysis["endgame"]["black"]["std"] = np.std(end_moves) if len(end_moves) > 1 else 0
                    
                    all_games.append((row_idx, white_player, black_player, avg_cp_white, final_acc_white, avg_cp_black, final_acc_black, stage_analysis))
                    game_details.append({
                        "row_idx": row_idx,
                        "white_player": white_player,
                        "black_player": black_player,
                        "moves": move_details,
                        "summary": {
                            "white": {"cp_loss": avg_cp_white, "accuracy": final_acc_white},
                            "black": {"cp_loss": avg_cp_black, "accuracy": final_acc_black}
                        },
                        "stage_analysis": stage_analysis
                    })
                else:
                    all_games.append((row_idx, white_player, black_player, None, None, None, None, None))
                    skipped_indices.append(row_idx)
            
            except Exception as e:
                if is_verbose:
                    print(f"Game {row_idx+1}: Error analyzing game: {e}")
                    traceback.print_exc()
                all_games.append((row_idx, white_player, black_player, None, None, None, None, None))
                skipped_indices.append(row_idx)
            
            pbar.update(1)
        pbar.close()
    
    if skipped_indices and is_verbose:
        print(f"\n跳过/分析失败的棋局数: {len(skipped_indices)}")
    
    return all_games, game_details


def analyze_csv(csv_file_path, engine_path, threads, depth, is_verbose=False):
    """
    分析CSV文件中的国际象棋对局，计算每局对局的准确率和CP损失，
    并将结果添加到原始DataFrame中
    
    input:
    csv_file_path (str) 
    engine_path (str) 
    threads (int) 
    depth (int)
    is_verbose (bool)
    
    output:
    tuple: (result_dict, original_df with analysis result)
    """
    # 读取CSV到DataFrame
    with open(csv_file_path, 'r', encoding='utf-8') as f:
        first_line = f.readline().strip()
        delimiter = '\t' if '\t' in first_line else ','
    original_df = pd.read_csv(csv_file_path, delimiter=delimiter)
    
    engine = SimpleStockfishEngine(engine_path, threads)
    
    try:
        all_games, game_details = process_csv(csv_file_path, engine, depth, is_verbose)
        
        # 创建与原始DataFrame长度相同的空列表
        white_cp_loss_list = [None] * len(original_df)
        white_accuracy_list = [None] * len(original_df)
        black_cp_loss_list = [None] * len(original_df)
        black_accuracy_list = [None] * len(original_df)
        
        # 添加阶段分析结果的列表
        white_beginning_acc = [None] * len(original_df)
        white_beginning_std = [None] * len(original_df)
        white_middle_acc = [None] * len(original_df)
        white_middle_std = [None] * len(original_df)
        white_endgame_acc = [None] * len(original_df)
        white_endgame_std = [None] * len(original_df)
        
        black_beginning_acc = [None] * len(original_df)
        black_beginning_std = [None] * len(original_df)
        black_middle_acc = [None] * len(original_df)
        black_middle_std = [None] * len(original_df)
        black_endgame_acc = [None] * len(original_df)
        black_endgame_std = [None] * len(original_df)
        
        # 有效对局数据统计
        valid_games = 0
        total_avg_cp_white = total_avg_cp_black = 0.0
        total_acc_white = total_acc_black = 0.0
        
        for game in all_games:
            row_idx, white, black, avg_cp_white, acc_white, avg_cp_black, acc_black, stage_analysis = game
            
            # 将结果填入对应索引位置
            white_cp_loss_list[row_idx] = avg_cp_white
            white_accuracy_list[row_idx] = acc_white
            black_cp_loss_list[row_idx] = avg_cp_black
            black_accuracy_list[row_idx] = acc_black
            
            # 填充阶段分析结果
            if stage_analysis is not None:
                # 白方
                white_beginning_acc[row_idx] = stage_analysis["beginning"]["white"]["accuracy"]
                white_beginning_std[row_idx] = stage_analysis["beginning"]["white"]["std"]
                white_middle_acc[row_idx] = stage_analysis["middle"]["white"]["accuracy"]
                white_middle_std[row_idx] = stage_analysis["middle"]["white"]["std"]
                white_endgame_acc[row_idx] = stage_analysis["endgame"]["white"]["accuracy"]
                white_endgame_std[row_idx] = stage_analysis["endgame"]["white"]["std"]
                
                # 黑方
                black_beginning_acc[row_idx] = stage_analysis["beginning"]["black"]["accuracy"]
                black_beginning_std[row_idx] = stage_analysis["beginning"]["black"]["std"]
                black_middle_acc[row_idx] = stage_analysis["middle"]["black"]["accuracy"]
                black_middle_std[row_idx] = stage_analysis["middle"]["black"]["std"]
                black_endgame_acc[row_idx] = stage_analysis["endgame"]["black"]["accuracy"]
                black_endgame_std[row_idx] = stage_analysis["endgame"]["black"]["std"]
            
            # 只统计有效对局
            if avg_cp_white is not None and acc_white is not None and avg_cp_black is not None and acc_black is not None:
                valid_games += 1
                total_avg_cp_white += avg_cp_white
                total_avg_cp_black += avg_cp_black
                total_acc_white += acc_white
                total_acc_black += acc_black
        
        # 添加新列到原始DataFrame
        original_df['White CP Loss'] = white_cp_loss_list
        original_df['White Accuracy'] = white_accuracy_list
        original_df['Black CP Loss'] = black_cp_loss_list
        original_df['Black Accuracy'] = black_accuracy_list
        
        # 添加阶段分析列
        original_df['White Beginning Accuracy'] = white_beginning_acc
        original_df['White Beginning Std'] = white_beginning_std
        original_df['White Middle Accuracy'] = white_middle_acc
        original_df['White Middle Std'] = white_middle_std
        original_df['White Endgame Accuracy'] = white_endgame_acc
        original_df['White Endgame Std'] = white_endgame_std
        
        original_df['Black Beginning Accuracy'] = black_beginning_acc
        original_df['Black Beginning Std'] = black_beginning_std
        original_df['Black Middle Accuracy'] = black_middle_acc
        original_df['Black Middle Std'] = black_middle_std
        original_df['Black Endgame Accuracy'] = black_endgame_acc
        original_df['Black Endgame Std'] = black_endgame_std
        
        # 输出统计信息
        print(f"\n总棋局数: {len(original_df)}")
        print(f"成功分析棋局数: {valid_games}")
        print(f"跳过/分析失败棋局数: {len(original_df) - valid_games}")
        
        summary = {}
        if valid_games > 0:
            print("\n===== 总体统计 =====")
            print(f'有效分析对局数: {valid_games}')
            print(f'白方平均: {total_avg_cp_white/valid_games:.1f}cp损失, {total_acc_white/valid_games:.1f}%准确率')
            print(f'黑方平均: {total_avg_cp_black/valid_games:.1f}cp损失, {total_acc_black/valid_games:.1f}%准确率')
            
            summary = {
                "total_games": valid_games,
                "avg_white_cp_loss": total_avg_cp_white/valid_games,
                "avg_white_accuracy": total_acc_white/valid_games,
                "avg_black_cp_loss": total_avg_cp_black/valid_games,
                "avg_black_accuracy": total_acc_black/valid_games
            }
        
        result_dict = {
            "game_results": game_details,  # 使用详细游戏信息替换简单结果
            "summary": summary
        }
        
        return result_dict, original_df
    
    finally:
        engine.quit()

In [5]:
csv_file_path="C:\\Users\\Administrator\\Desktop\\2014.10-15.2\\results\\aftermerge\\1410-1503.csv"
engine_path='C:\\Users\\Administrator\\Downloads\\stockfish-windows-x86-64-avx2\\stockfish\\stockfish-windows-x86-64-avx2.exe'
threads = 2  
depth = 16  
is_verbose = True  

# 运行分析
result_dict,mediate_df = analyze_csv(csv_file_path, engine_path, threads, depth, is_verbose)

# 创建对局结果DataFrame
display(mediate_df)

分析棋局:   0%|          | 0/1686 [00:00<?, ?it/s]


—— Analyzing Game 1 ——
White: VovAn1991 (2355), Black: RealBlindMuddy (2037)

—— Analyzing Game 2 ——
White: TomsKantans (2234), Black: lorcho (2632)

—— Analyzing Game 3 ——
White: ilmago (1718), Black: GMG (2278)


分析棋局:   0%|          | 2/1686 [00:04<1:07:01,  2.39s/it]


—— Analyzing Game 4 ——
White: SabinaFoisor (2284), Black: swayamsm (2288)


分析棋局:   0%|          | 3/1686 [00:06<56:47,  2.02s/it]  

...

分析棋局:  97%|█████████▋| 1642/1686 [5:01:54<13:58, 19.06s/it]


—— Analyzing Game 1686 ——
White: GGuseinov (2521), Black: AlexanderL (2533)


分析棋局:  97%|█████████▋| 1643/1686 [5:02:15<07:54, 11.04s/it]


跳过/分析失败的棋局数: 43

总棋局数: 1686
成功分析棋局数: 1634
跳过/分析失败棋局数: 52

===== 总体统计 =====
有效分析对局数: 1634
白方平均: 33.3cp损失, 86.8%准确率
黑方平均: 34.1cp损失, 86.5%准确率


,Event,Site,Date,Round,White,Black,Result,WhiteElo,BlackElo,TimeControl,...,White Middle Accuracy,White Middle Std,White Endgame Accuracy,White Endgame Std,Black Beginning Accuracy,Black Beginning Std,Black Middle Accuracy,Black Middle Std,Black Endgame Accuracy,Black Endgame Std
0,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,VovAn1991,RealBlindMuddy,1-0,2355,2037,180+2,...,NaN,NaN,97.355130,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,TomsKantans,lorcho,0-1,2234,2632,180+2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,ilmago,GMG,0-1,1718,2278,180+2,...,82.985401,16.838992,77.517956,20.594522,97.275020,3.716128,90.504206,9.857504,89.741507,16.832719
3,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,SabinaFoisor,swayamsm,1-0,2284,2288,180+2,...,93.624434,5.337049,93.291736,3.030791,93.303910,2.470164,90.883089,2.193190,96.366502,3.633498
4,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,KingPetrosian,mat_kolosowski,0-1,2095,2352,180+2,...,93.308720,12.892787,99.687454,0.740533,99.355770,1.274435,98.939505,2.150934,99.886281,0.270958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,Mugzyyy,megarompa,0-1,2449,2683,180+2,...,92.094203,11.973641,85.556916,23.677186,92.783882,9.567825,90.203439,14.422086,88.992977,20.332599
1682,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,1stSecond,AndreyOstrovskiy,1/2-1/2,2372,2357,180+2,...,94.140987,14.125250,99.315577,2.138444,90.751940,14.598213,96.391597,10.093465,99.369315,2.715192
1683,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,ChessPsych0,SuPeR_Fm,0-1,2065,2341,180+2,...,97.506019,6.503138,93.914429,17.383642,91.750301,10.206618,98.488577,3.233310,96.504535,12.443981
1684,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,Fandorine,Tilicheev_Viacheslav,1/2-1/2,2566,2519,180+2,...,97.803933,4.436912,97.627445,10.923771,93.973668,6.090862,98.436069,2.892443,97.454330,11.399091


In [6]:
display(mediate_df)

,Event,Site,Date,Round,White,Black,Result,WhiteElo,BlackElo,TimeControl,...,White Middle Accuracy,White Middle Std,White Endgame Accuracy,White Endgame Std,Black Beginning Accuracy,Black Beginning Std,Black Middle Accuracy,Black Middle Std,Black Endgame Accuracy,Black Endgame Std
0,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,VovAn1991,RealBlindMuddy,1-0,2355,2037,180+2,...,NaN,NaN,97.355130,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
1,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,TomsKantans,lorcho,0-1,2234,2632,180+2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,ilmago,GMG,0-1,1718,2278,180+2,...,82.985401,16.838992,77.517956,20.594522,97.275020,3.716128,90.504206,9.857504,89.741507,16.832719
3,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,SabinaFoisor,swayamsm,1-0,2284,2288,180+2,...,93.624434,5.337049,93.291736,3.030791,93.303910,2.470164,90.883089,2.193190,96.366502,3.633498
4,*** Monthly 3|2 Blitz Masters,Chess.com,2014.10.28,1,KingPetrosian,mat_kolosowski,0-1,2095,2352,180+2,...,93.308720,12.892787,99.687454,0.740533,99.355770,1.274435,98.939505,2.150934,99.886281,0.270958
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,Mugzyyy,megarompa,0-1,2449,2683,180+2,...,92.094203,11.973641,85.556916,23.677186,92.783882,9.567825,90.203439,14.422086,88.992977,20.332599
1682,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,1stSecond,AndreyOstrovskiy,1/2-1/2,2372,2357,180+2,...,94.140987,14.125250,99.315577,2.138444,90.751940,14.598213,96.391597,10.093465,99.369315,2.715192
1683,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,ChessPsych0,SuPeR_Fm,0-1,2065,2341,180+2,...,97.506019,6.503138,93.914429,17.383642,91.750301,10.206618,98.488577,3.233310,96.504535,12.443981
1684,*** Titled Tuesday 3|2 Blitz,Chess.com,2015.03.03,9,Fandorine,Tilicheev_Viacheslav,1/2-1/2,2566,2519,180+2,...,97.803933,4.436912,97.627445,10.923771,93.973668,6.090862,98.436069,2.892443,97.454330,11.399091


In [7]:
def simplify_termination(term):
    if pd.isna(term) or term == '':
        return 'None'
    #WON
    if 'resignation' in term.lower():
        return 'resignation'
    elif 'checkmate' in term.lower():
        return 'checkmate'
    elif 'game abandoned' in term.lower():
        return 'game abandoned'
    elif 'time' in term.lower() and not ('insufficient' in term.lower() or 'stalemate' in term.lower()):  # 修正拼写错误
        return 'time'
    #DRAW
    elif 'repetition' in term.lower():
        return 'repetition'
    elif 'agreement' in term.lower():
        return 'agreement'
    elif 'insufficient material' in term.lower():
        return 'insufficient material'
    elif 'stalemate' in term.lower():
        return 'stalemate'
    elif '50-move rule' in term.lower():
        return '50-move rule'
    elif 'timeout vs insufficient material' in term.lower():
        return 'timeout vs insufficient material'
    else:
        return term
    
mediate_df['Termination']=mediate_df['Termination'].apply(simplify_termination)

In [8]:
mediate_df.to_csv('1410-1503results.csv',index=False)
#C:\Users\Administrator\Desktop\simple eda\simple eda\EDA\1410-1503results.csv